In [1]:
import zipfile
import os

zip_path = "/content/Face Detection.v1i.yolov8.zip"
extract_path = "/content/face_dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

os.listdir(extract_path)


['train', 'README.dataset.txt', 'data.yaml', 'README.roboflow.txt', 'test']

In [2]:
!pip install -U ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.4 MB/s eta 0:00:00


In [4]:
!ls /content/face_dataset


data.yaml  README.dataset.txt  README.roboflow.txt  test  train


In [5]:
!mkdir -p /content/face_dataset/valid/images
!mkdir -p /content/face_dataset/valid/labels


In [6]:
import os, random, shutil

train_img = "/content/face_dataset/train/images"
train_lbl = "/content/face_dataset/train/labels"
val_img   = "/content/face_dataset/valid/images"
val_lbl   = "/content/face_dataset/valid/labels"

images = os.listdir(train_img)
random.shuffle(images)

val_count = int(0.15 * len(images))
val_images = images[:val_count]

for img in val_images:
    shutil.move(f"{train_img}/{img}", f"{val_img}/{img}")
    shutil.move(f"{train_lbl}/{img.replace('.jpg','.txt')}",
                f"{val_lbl}/{img.replace('.jpg','.txt')}")


In [7]:
from ultralytics import YOLO

model = YOLO("yolov8m.pt")

model.train(
    data="/content/face_dataset/data.yaml",
    epochs=80,
    imgsz=640,
    batch=16,
    optimizer="AdamW",
    lr0=0.001,
    weight_decay=0.0005,
    patience=15,
    mosaic=1.0,
    scale=0.5,
    fliplr=0.5,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    device=0
)


Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/face_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=15, perspective=0.0, plots=True, pose

KeyboardInterrupt: 

In [9]:
!cp /content/runs/detect/train2/weights/best.pt \
    /content/yolov8m_multiscale_face.pt



In [10]:
from google.colab import files
files.download("/content/yolov8m_multiscale_face.pt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
metrics = model.val()
print(metrics)


In [ ]:
model.predict(
    source="/content/face_dataset/test/images",
    conf=0.4,
    save=True
)
